# Demo der Klassifizierung auf Basis des RASA-Stacks
Das nachfolgende Notebook soll zum vertesten der Klassifizierung dienen und eine Aussagen ueber die Qualitaet der gebildeten Klassen liefern.

## Anleitung
Solande dieses Stystem nicht lokal verwendet wird, kann das Kapitel 1 "Installation" uebersprungen werden.

Es kann also direkt mit dem Kapitel 2 "Trainieren der Intents" gestartet werden

## 1. Installation
Damit alle notwendigen Pakete installiert sind muessen folgende Pakete installiert werden.
```
pip install rasa_core
pip install rasa_nlu
pip install spacy
pip install sklearn_crfsuite
spacy download de_core_news_sm
spacy link de_core_news_sm de --force
```
Im Anschluss kann die Installtion mit der naechsten Zelle getestet werden

In [4]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("de")("Deutsches Sprach-Model geladen"))

rasa_nlu: 0.15.1 rasa_core: 0.14.5
Loading spaCy language model...
Deutsches Sprach-Model geladen


## 2. Trainieren der Intents
Nachfolgend koennen die Intents innerhalb der Anfuehrungszeichen gepflegt werden. Da gilt folgende Notation;
```
## intent:<Name des Anliegens>
- <Beispiel 1 fuer das Anliegen>
- <Beispiel 2 fuer das Anliegen>
- ...

## intent:<Name eines weitern Anliegens>
- <Beispiel 1 fuer das Anliegen>
- <Beispiel 2 fuer das Anliegen>
- ...

...
```

Beispiel:
```
## intent:hallo
- Hallo
- Guten Tag
- Guten Morgen
- Hallo zusammen

## intent:wetter
- Wie ist das Wetter?
- Wird es regnen?
- Scheint gerade die Sonne?

...
```

Je mehr Beispiele fuer ein Anliegne gegeben werden, desto besser kann das Model trainiert werden und auch spaeter auf Eingaben reagieren.

Wenn alle Daten eingegeben oder entsprechend angepasst wurden, dann die beiden Tasten `<STRG>+<ENTER>` gleichzeitig druecken, um die Aenderungen zu speichern.

Dies kann einen Moment dauern und es koennten auch rote Meldungen erscheinen. Diese koennen vorerst ignoriert werden. Das Training ist fertig, wenn unterhalb des naechsten Feldes _Taining beendet!_ erscheint.

In [13]:
nlu_md = """

## intent:asu
- ich wurde fristlos gekuendigt, was ist zu tun?
- ich wurde gekuendigt und moechte mich gerne arbeitslos melden
- ich bin ab kommender Woche arbeitslos und weiss nicht was ich machen soll
- heute habe ich eine kuendigung erhalten und melde mich hiermit
- mein betrieb hat mich gekuendigt, was soll ich jetzt machen?
- wie kann ich mich arbeitssuchend melden?
- was ist zu tun, wenn mir mein chef gerade gekuendigt hat
- ich wurde gerade rausgeworfen, ich bitte um hilfe

## intent:set
- ich habe mein abitur und weiss nicht was ich machen soll
- ich brauche beratung welche ausbildung ich angehen soll
- ich brauche hilfe beim finden meines Studiums
- ich weiss nicht, was ich studieren soll
- ich habe keine orientierung nach meinem Schulabschluss
- ich weiss nicht, ob und was ich studieren soll
- soll ich nach dem Abitur eine Ausbildung oder ein Stium machen
- ich habe keine vorstellung, welchen Studiengang ich wqehlen soll


## intent:zahlung
- wo ist mein geld?
- ich habe diesen Monat noch kein Geld erhalten
- bis jetzt gab es keine Zahlung
- Schon wieder gibt es Probleme mit dem Zahlungseingang
- Ich habe kein geld erhalten
- Seit Tagen fehlt schon das Geld, es wird dringend gebraucht

"""

# bitte nur den oberen Teil bearbeiten

%store nlu_md > nlu.md

config = """
language: "de"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_crf"
- name: "ner_synonyms"
""" 

%store config > config.yml

from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

print("Taining beendet!")

Writing 'nlu_md' (str) to file 'nlu.md'.
Writing 'config' (str) to file 'config.yml'.
Fitting 2 folds for each of 6 candidates, totalling 12 fits
Taining beendet!


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


## 3. Abfragen der Anliegen fuer neue Texte
In der nachfolgenden Zeile bitte nach der initialen Erstellung der oberen Anliegen oder nach Aenderungen dieser erneut `<STRG>+<ENTER>` gleichzeitig druecken. Dann erscheint ein neues Eingabefeld. Den neuen Kundentext in diesem Feld eingeben und mit `<ENTER>` absenden. Als Ergebnis erscheint der Name des Anliegens und die Zuverlaessigkeit (convidence), mit welcher die Anfrage zugeordnet werden konnte.

In [15]:
print("Neue Anfragen eingeben oder mit dem Wort 'stop' beenden")
while True:
    a = input()
    if a == 'stop':
        break
    responses = interpreter.parse(a)
    print(responses['intent'])

Neue Anfragen eingeben oder mit dem Wort 'stop' beenden
Ich habe diesen Monat kein geld erhalten
{'name': 'zahlung', 'confidence': 0.7202977609711297}
Ich wurde gekuendigt, was soll ich tun?
{'name': 'asu', 'confidence': 0.6448943019307292}
Ich habe keine Ahnung was ich studieren soll
{'name': 'set', 'confidence': 0.6281458600273484}
stop
